### Dataset Description

The dataset for this project is the Digit Description dataset from Kaggle. Each image is 28 pixels by 28 pixels (784 total pixels) and has a digit between 0-9. Each pixel has a single value between 0 and 255 (inclusive) associated with it indicating lightness or darkness, with higher numbers meaning darker. There are two datasets; train.csv and test.csv. 

The training dataset has 785 columns. The first column, 'label,' is the digit the picture encodes for. The rest of the columns contain the pixel-values of the associated image. Each row corresponds to a different image. Each pixel in the training set has a name like pixelx, where x is an integer between 0 and 783 (inclusive). Each column representing a pixel (discluding label column) has a name pixelx where x is decomposed as `x = i*28 + j`, where i and j are integers between 0 and 27 representing the pixel row and column of the image. The test dataset (28000 images) is the same as the training set, except it does not contain the 'label' column.

### Goal

For each of the 28000 images in the test set, the output should be a single line containing the imageId and the predicted digit. The evaluation metric is the categorization accuracy, or the proportion of test images that are correctly classified.

In [1]:
# pandas is for reading data
import pandas as pd
# numpy is for linear algebra
import numpy as np
from matplotlib import pyplot as plt
;


C:\Users\dylan\AppData\Local\Temp\ipykernel_3296\1437040094.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


''

In [2]:
data = pd.read_csv("data_files/train.csv")

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [5]:
# don't want to be working with pandas df
# we want to be working with numpy arrays so we can perform linear algebra manipulations

data = np.array(data)

In [6]:
m,n = data.shape

# In order to avoid overfitting, we want to randomize the data and then split it into train and dev
np.random.shuffle(data)

# transpose the data so that each column is a row (easier)
data_dev = data[0:1000].T
# each column is now an image
# first row is now labels
# following rows (783) are each pixel
Y_dev = data_dev[0]
X_dev = data_dev[1:]

data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:]



In [16]:
data_train.size

32185000

In [19]:
data_train.shape

(785, 41000)

In [8]:
def init_params():
    # random.randn makes dist between -.5 and .5
    # wi is the weight vector (# second layer neurons, # first layer neurons)
        # each input neuron (784) connects to each output neuron (10)
    # bi is the bias in the output layer neurons
    w1 = np.random.randn(10,784)
    b1 = np.random.randn(10,1)
    w2 = np.random.randn(10,10)
    b2 = np.random.randn(10,1)
    return w1, b1, w2, b2

In [11]:
w1, b1, w2, b2 = init_params()

In [14]:
def ReLU(Z):
    # maximum is element-wise so it runs that calc for each element in Z 
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

In [15]:
# left off at 16:54
def forward_prop(w1, b1, w2, b2, x):
    z1 = w1.dot(X) + b1
    A1 = ReLU(z1)
    z2 = w2.dot(A1) + b2
    A2 = softmax(z2)
    return z1, A1, z2, A2

In [20]:
# transform a vector Y of class labels into a one-hot encoded matrix
# one-hot encoding is a common way to represent categorical variables as binary vectors 
# Y is going to be an array (mx1) where each element is the predicted class for the equivalent instance column of the input data array
def one_hot(Y):
    # np.zeros line creates a 2D array of zeros with shape determined by number of samples and number of unique classes   
        # y.size returns the total number of elements in Y which represents the number of samples or instances
        # Y.max() + 1 calculates the max value in Y and adds 1 to determine the number of unique classes
            # adding one is necessary because the classes start from 0 (0-9)
    ohY = np.zeros((Y.size, Y.max()+1))
    # "for each row, go to the column specified by the label in Y and set it equal to 1"
    # by indexing ohY like this, we are effectively selecting one position per row, determined by the class label in Y
    # each row in ohY corresponds to a sample in Y and each column in ohY corresponds to a class
    # for each row in ohY, the column corresponding to its class label is set to 1 (all other columns remain 0)
        #  np.arange(Y.size) generates an array of indices from 0 to Y.size - 1 corresponding to each sample in Y --> specifies what row to access
        # Y contains the class label for each sample
            # when used as an index, Y selects the column in ohY that corresponds to its class label
    # 
    ohY[np.arange(Y.size), Y] = 1
    # transpose because we want each column to be a sample not each row
    return ohY.T

def deriv_ReLU(Z):
    # relu has deriv of 1 for x > 0 (because x = x) and 0 for x <=0 (because x = 0)
    # this works because booleans are converted to 1 for true and 0 for false so if a number is positive then its deriv was 1
    # since
    return Z > 0

In [ ]:
# !!!! print out the shape of all of these matrices as they are made to make sure sizing is correct for all the matmults

In [21]:
def back_prop(z1, A1, z2, A2, w2, X, Y):
    m = Y.size
    ohY = one_hot(Y)
    dz2 = A2 - ohY
    dw2 = 1/m * dz2.dot(A1.T)
    db2 = 1/m * np.sum(dz2)
    # I don't understand this next part
    dz1 = w2.T.dot(dz2) * deriv_ReLU(z1)
    dw1 = 1/m * dz1.dot(X.T)
    db1 = 1/m * np.sum(dz1)
    return dz2, dw2, db2, dz1, dw1, db1

In [22]:
def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, lr):
    w1 = w1 - lr*dw1
    b1 = b1 - lr*db1
    w2 = w2 - lr*dw2
    b2 = b2 - lr*db2
    return w1, b1, w2, b2
